In [22]:
import os
import pandas as pd
from dotenv import load_dotenv

In [15]:
#Definindo funções de filtragem

def create_dtypes(all_columns,type):
    dtypes = {}
    for col in all_columns:
        dtypes[col] = type
    return dtypes

def drop_columns(df, columns_to_drop):
    df = df.drop(columns=columns_to_drop)
    return df

def wanted_values(df,col, values):
    df = df[df[col].isin(values)]
    return df

def wanted_single_value(df, col, value):
    df = df[df[col] == value]
    return df

def numeric_columns_drop_nas(df, numeric_columns):
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].fillna(0)
    return df
    
def convert_column_to_datetime(df,date_columns):
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors = 'coerce')
    return df



In [12]:
#Configurando paramêtros de leitura

# Aplicando as funções e filtrando os dados
load_dotenv()

chunksize = 10**6  # Tamanho dos chunks para leitura

filepath =  os.getenv('CSV_RAW_PATH')

# Definir as colunas da tabela em uma lista
columns_list = ['id', 'title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend', 'streams', 'track_id', 'album', 'popularity', 'duration_ms', 'explicit', 'release_date', 'available_markets', 'af_danceability', 'af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness', 'af_acousticness', 'af_instrumentalness', 'af_liveness', 'af_valence', 'af_tempo', 'af_time_signature']

#Definindo uma lista vazia para guardar os chunks
filtered_data = []

# Obter o dicionario dtypes
dtypes = create_dtypes(columns_list,type=str)
print(dtypes)







{'id': <class 'str'>, 'title': <class 'str'>, 'rank': <class 'str'>, 'date': <class 'str'>, 'artist': <class 'str'>, 'url': <class 'str'>, 'region': <class 'str'>, 'chart': <class 'str'>, 'trend': <class 'str'>, 'streams': <class 'str'>, 'track_id': <class 'str'>, 'album': <class 'str'>, 'popularity': <class 'str'>, 'duration_ms': <class 'str'>, 'explicit': <class 'str'>, 'release_date': <class 'str'>, 'available_markets': <class 'str'>, 'af_danceability': <class 'str'>, 'af_energy': <class 'str'>, 'af_key': <class 'str'>, 'af_loudness': <class 'str'>, 'af_mode': <class 'str'>, 'af_speechiness': <class 'str'>, 'af_acousticness': <class 'str'>, 'af_instrumentalness': <class 'str'>, 'af_liveness': <class 'str'>, 'af_valence': <class 'str'>, 'af_tempo': <class 'str'>, 'af_time_signature': <class 'str'>}


In [13]:
# Definindo os paramtros de filtragem

columns_to_drop = [ 'url', 'available_markets']

wanted_regions = ['Global', 'Brazil', 'United States', 'United Kingdom', 'Mexico', 'Germany', 'France', 'Spain', 'Bolivia', 'Chile', 'Argentina', 'Colombia']

chart_value = 'top200'

numeric_cols = ['rank', 'streams', 'popularity', 'duration_ms', 'explicit', 'af_danceability', 'af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness', 'af_acousticness', 'af_instrumentalness', 'af_liveness', 'af_valence', 'af_tempo', 'af_time_signature']

date_cols = ['date','release_date']

In [16]:
#Executando o processo

for chunk in pd.read_csv(filepath, chunksize=chunksize, dtype=dtypes, low_memory=False):

    chunk = drop_columns(chunk, columns_to_drop)
    chunk =  wanted_values(chunk,'region', wanted_regions)
    chunk = wanted_single_value(chunk, 'chart', 'top200')
    chunk = numeric_columns_drop_nas(chunk,numeric_cols)
    chunk = convert_column_to_datetime(chunk, date_cols)
    
    filtered_data.append(chunk)

filtered_df = pd.concat(filtered_data)

In [17]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4277997 entries, 0 to 25276606
Data columns (total 27 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Unnamed: 0           int64         
 1   title                object        
 2   rank                 int64         
 3   date                 datetime64[ns]
 4   artist               object        
 5   region               object        
 6   chart                object        
 7   trend                object        
 8   streams              float64       
 9   track_id             object        
 10  album                object        
 11  popularity           float64       
 12  duration_ms          float64       
 13  explicit             float64       
 14  release_date         datetime64[ns]
 15  af_danceability      float64       
 16  af_energy            float64       
 17  af_key               float64       
 18  af_loudness          float64       
 19  af_mode              floa

In [18]:
filtered_df.head()

,Unnamed: 0,title,rank,date,artist,region,chart,trend,streams,track_id,...,af_key,af_loudness,af_mode,af_speechiness,af_acousticness,af_instrumentalness,af_liveness,af_valence,af_tempo,af_time_signature
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,Argentina,top200,SAME_POSITION,253019.0,6mICuAdrwEjh6Y6lroV2Kg,...,8.0,-2.921,0.0,0.0776,0.18700,0.000030,0.1590,0.907,102.034,4.0
1,1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,Argentina,top200,MOVE_UP,223988.0,7DM4BPaS7uofFul3ywMe46,...,11.0,-4.070,0.0,0.2260,0.00431,0.000017,0.1010,0.533,99.935,4.0
2,2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,Argentina,top200,MOVE_DOWN,210943.0,3AEZUABDXNtecAOSC1qTfo,...,4.0,-3.073,0.0,0.0502,0.40000,0.000000,0.1760,0.710,93.974,4.0
3,3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",Argentina,top200,SAME_POSITION,173865.0,6rQSrBHf7HlZjtcMZ4S4bO,...,0.0,-4.361,1.0,0.3260,0.55100,0.000003,0.1260,0.555,180.044,4.0
4,4,Shaky Shaky,5,2017-01-01,Daddy Yankee,Argentina,top200,MOVE_UP,153956.0,58IL315gMSTD37DOZPJ2hf,...,6.0,-4.228,0.0,0.2920,0.07600,0.000000,0.0631,0.873,88.007,4.0


In [19]:
filtered_df = filtered_df.replace('"', ' ', regex=True)
filtered_df.head(15)

,Unnamed: 0,title,rank,date,artist,region,chart,trend,streams,track_id,...,af_key,af_loudness,af_mode,af_speechiness,af_acousticness,af_instrumentalness,af_liveness,af_valence,af_tempo,af_time_signature
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,Argentina,top200,SAME_POSITION,253019.0,6mICuAdrwEjh6Y6lroV2Kg,...,8.0,-2.921,0.0,0.0776,0.18700,0.000030,0.1590,0.907,102.034,4.0
1,1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,Argentina,top200,MOVE_UP,223988.0,7DM4BPaS7uofFul3ywMe46,...,11.0,-4.070,0.0,0.2260,0.00431,0.000017,0.1010,0.533,99.935,4.0
2,2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,Argentina,top200,MOVE_DOWN,210943.0,3AEZUABDXNtecAOSC1qTfo,...,4.0,-3.073,0.0,0.0502,0.40000,0.000000,0.1760,0.710,93.974,4.0
3,3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",Argentina,top200,SAME_POSITION,173865.0,6rQSrBHf7HlZjtcMZ4S4bO,...,0.0,-4.361,1.0,0.3260,0.55100,0.000003,0.1260,0.555,180.044,4.0
4,4,Shaky Shaky,5,2017-01-01,Daddy Yankee,Argentina,top200,MOVE_UP,153956.0,58IL315gMSTD37DOZPJ2hf,...,6.0,-4.228,0.0,0.2920,0.07600,0.000000,0.0631,0.873,88.007,4.0
5,5,Traicionera,6,2017-01-01,Sebastian Yatra,Argentina,top200,MOVE_DOWN,151140.0,5J1c3M4EldCfNxXwrwt8mT,...,11.0,-4.933,1.0,0.0638,0.14200,0.000000,0.2190,0.661,91.012,4.0
6,6,Cuando Se Pone a Bailar,7,2017-01-01,Rombai,Argentina,top200,MOVE_DOWN,148369.0,1MpKZi1zTXpERKwxmOu1PH,...,11.0,-7.169,0.0,0.1730,0.08510,0.000027,0.0840,0.937,205.643,4.0
7,7,Otra vez (feat. J Balvin),8,2017-01-01,Zion & Lennox,Argentina,top200,MOVE_DOWN,143004.0,3QwBODjSEzelZyVjxPOHdq,...,10.0,-5.429,1.0,0.1000,0.05590,0.000486,0.4400,0.704,96.016,4.0
8,8,La Bicicleta,9,2017-01-01,"Carlos Vives, Shakira",Argentina,top200,MOVE_UP,126389.0,0sXvAOmXgjR2QUqLK1MltU,...,0.0,-2.147,1.0,0.1290,0.19800,0.000002,0.3360,0.953,179.935,4.0
9,9,Dile Que Tu Me Quieres,10,2017-01-01,Ozuna,Argentina,top200,MOVE_DOWN,112012.0,20ZAJdsKB5IGbGj4ilRt2o,...,1.0,-6.682,1.0,0.0782,0.09980,0.000000,0.0679,0.825,175.914,4.0


In [28]:
# Exportando a base processada em csv
load_dotenv()
output_path = os.getenv('PROCESSED_CSV_PATH')
filtered_df.to_csv(output_path, index=False)